Components

1.Hardware:

Ultrasonic sensor (e.g., HC-SR04) to detect car presence.
ESP8266/Arduino for processing and communication.
LEDs or buzzer for indicating parking spot status.

2.Software:

Arduino IDE for sensor programming.
Python for data collection and ML.
Libraries: pandas, scikit-learn, matplotlib.


Hardware Setup

Connection of HC-SR04 Sensor
VCC → 5V on ESP8266/Arduino.
GND → Ground.
TRIG → Digital pin D4.
ECHO → Digital pin D5.



Arduino Code:

The Arduino code reads parking availability using the ultrasonic sensor.

In [ ]:
#define TRIG_PIN D4
#define ECHO_PIN D5

void setup() {
  Serial.begin(9600);
  pinMode(TRIG_PIN, OUTPUT);
  pinMode(ECHO_PIN, INPUT);
}

void loop() {
  digitalWrite(TRIG_PIN, LOW);
  delayMicroseconds(2);
  digitalWrite(TRIG_PIN, HIGH);
  delayMicroseconds(10);
  digitalWrite(TRIG_PIN, LOW);

  long duration = pulseIn(ECHO_PIN, HIGH);
  float distance = (duration * 0.034) / 2;  // Convert to cm

  // If distance is less than 10 cm, parking spot is occupied
  if (distance < 10) {
    Serial.println("Occupied");
  } else {
    Serial.println("Available");
  }

  delay(1000);  // Check every second
}


5. Collecting Data
Python Script for Data Collection
The Arduino sends data (e.g., "Occupied" or "Available") via serial communication. Use Python to collect and save it for analysis.

In [ ]:
#Install Required Libraries
pip install pyserial pandas


In [ ]:
import serial
import pandas as pd
import time

# Configure serial port
serial_port = '/dev/ttyUSB0'  # Replace with your port (COM3 for Windows)
baud_rate = 9600
arduino = serial.Serial(serial_port, baud_rate)

# Data collection variables
data = []
start_time = time.time()

print("Collecting parking data... Press Ctrl+C to stop.")

try:
    while True:
        if arduino.in_waiting > 0:
            status = arduino.readline().decode('utf-8').strip()
            timestamp = time.time() - start_time
            print(f"Time: {timestamp:.2f}s, Status: {status}")
            data.append({"Time (s)": timestamp, "Status": status})
except KeyboardInterrupt:
    print("Data collection stopped.")

# Save the data
df = pd.DataFrame(data)
df.to_csv("parking_data.csv", index=False)
print("Data saved to parking_data.csv")


6. Training the ML Model
Use the collected data to train a model predicting parking availability trends.

Python Code for ML
Install Libraries

In [ ]:
pip install scikit-learn matplotlib


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

# Load collected data
data = pd.read_csv("parking_data.csv")

# Prepare data for ML
data['Status'] = data['Status'].map({'Available': 0, 'Occupied': 1})  # Convert to binary
data['Time (min)'] = data['Time (s)'] // 60  # Group data into minutes
X = data[['Time (min)']]
y = data['Status']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Predict availability
future_time = [[10], [20], [30]]  # Predict for 10, 20, and 30 minutes
predictions = model.predict(future_time)

print("Predictions:")
for time, status in zip(future_time, predictions):
    status_text = "Available" if status == 0 else "Occupied"
    print(f"Time: {time[0]} minutes, Status: {status_text}")

# Visualize trends
plt.scatter(X, y, color='blue', label='Actual Data')
plt.plot(future_time, predictions, color='red', label='Predictions')
plt.xlabel('Time (min)')
plt.ylabel('Status (0 = Available, 1 = Occupied)')
plt.title('Parking Spot Prediction')
plt.legend()
plt.show()


7. Deployment
Real-Time Prediction
Integrate real-time data collection with predictions for live insights.

Example Prediction Code

In [ ]:
import joblib  # Save and load models

# Save the model
joblib.dump(model, 'parking_predictor.pkl')

# Load the model
model = joblib.load('parking_predictor.pkl')

# Real-time prediction
current_time = [[15]]  # Replace with real-time input (minutes)
prediction = model.predict(current_time)
status = "Available" if prediction[0] == 0 else "Occupied"
print(f"Current Parking Status: {status}")


8. How It Works
Hardware: Sensors detect parking availability.
Data Collection: Real-time data is logged.
ML Model: Predicts future parking availability based on trends.
Deployment: Use the model for live predictions and alert systems.